In [1]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

pd.set_option("display.max_columns", None)

load_dotenv()

SCORECARD_KEY = os.getenv("COLLEGE_SCORECARD_API_KEY")

In [2]:
BASE_URL = "https://api.data.gov/ed/collegescorecard/v1/schools"

Utilize the CSC data dictionary (Misc/CollegeScorecardDataDictionary.xlsx) to find variable names

In [9]:
fields = ",".join([
    "school.id",
    "school.name",
    "school.ownership",
    "school.main_campus",
    "programs.cip_4_digit.code",
    "programs.cip_4_digit.title",
    "programs.cip_4_digit.credential.level",
    "programs.cip_4_digit.distance",
    "latest.programs.cip_4_digit.earnings.5_yr.overall_median_earnings",
    "latest.programs.cip_4_digit.earnings.4_yr.working_not_enrolled.overall_count",
    "latest.programs.cip_4_digit.debt.staff_grad_plus.all.eval_inst.median"
])

params= {
    "api_key": SCORECARD_KEY,
    "school.state":"FL",
    "fields": fields,
    # "latest.programs.cip_4_digit.earnings.4_yr.overall_median_earnings__not":"null",
    "latest.programs.cip_4_digit.debt.staff_grad_plus.all.eval_inst.median__not":"null",
    "page":"1",
    "per_page":"20"
}

5 year meadian is not one of the features that allow feature

In [10]:
response = requests.get(
    BASE_URL,
    params = params,
)
data = response.json()
print(data)

{'metadata': {'page': 1, 'total': 217, 'per_page': 20}, 'results': [{'latest.programs.cip_4_digit': [{'code': '5106', 'title': 'Dental Support Services and Allied Professions.', 'distance': 1, 'credential': {'level': 1}, 'earnings': {'5_yr': {'overall_median_earnings': 32076}, '4_yr': {'working_not_enrolled': {'overall_count': 254}}}, 'debt': {'staff_grad_plus': {'all': {'eval_inst': {'median': 9500}}}}}, {'code': '5107', 'title': 'Health and Medical Administrative Services.', 'distance': 1, 'credential': {'level': 1}, 'earnings': {'5_yr': {'overall_median_earnings': 30683}, '4_yr': {'working_not_enrolled': {'overall_count': 41}}}, 'debt': {'staff_grad_plus': {'all': {'eval_inst': {'median': 9500}}}}}, {'code': '5108', 'title': 'Allied Health and Medical Assisting Services.', 'distance': 1, 'credential': {'level': 1}, 'earnings': {'5_yr': {'overall_median_earnings': 30099}, '4_yr': {'working_not_enrolled': {'overall_count': 216}}}, 'debt': {'staff_grad_plus': {'all': {'eval_inst': {'me

This is the code remove the json's nested format and normalize it into a df format

In [11]:
df = pd.json_normalize(
    data["results"],
    record_path=["latest.programs.cip_4_digit"],
    meta="school.name",
    errors="ignore"
)
df

,code,title,distance,credential.level,earnings.5_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,school.name
0,5106,Dental Support Services and Allied Professions.,1,1,32076.0,254.0,9500,Concorde Career Institute-Jacksonville
1,5107,Health and Medical Administrative Services.,1,1,30683.0,41.0,9500,Concorde Career Institute-Jacksonville
2,5108,Allied Health and Medical Assisting Services.,1,1,30099.0,216.0,9500,Concorde Career Institute-Jacksonville
3,5109,"Allied Health Diagnostic, Intervention, and Tr...",1,1,43170.0,73.0,15335,Concorde Career Institute-Jacksonville
4,5109,"Allied Health Diagnostic, Intervention, and Tr...",1,2,61133.0,58.0,20000,Concorde Career Institute-Jacksonville
...,...,...,...,...,...,...,...,...
450,4301,Criminal Justice and Corrections.,3,3,48574.0,71.0,13819,Indian River State College
451,4400,"Human Services, General.",1,3,39275.0,91.0,13500,Indian River State College
452,5138,"Registered Nursing, Nursing Administration, Nu...",1,2,61559.0,143.0,6500,Indian River State College
453,5138,"Registered Nursing, Nursing Administration, Nu...",3,3,93083.0,52.0,7250,Indian River State College


In [23]:
df.loc[df["earnings.5_yr.overall_median_earnings"].isna()]

,code,title,distance,credential.level,earnings.5_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,school.name
16,5138,"Registered Nursing, Nursing Administration, Nu...",0,2,NaN,38.0,23492,Concorde Career Institute-Tampa
19,5107,Health and Medical Administrative Services.,2,5,NaN,NaN,29325,AdventHealth University
22,5109,"Allied Health Diagnostic, Intervention, and Tr...",1,5,NaN,NaN,154604,AdventHealth University
24,5123,Rehabilitation and Therapeutic Professions.,0,6,NaN,NaN,168900,AdventHealth University
27,1104,Information Science/Studies.,0,2,NaN,NaN,25406,Florida Institute of Technology
...,...,...,...,...,...,...,...,...
440,5202,"Business Administration, Management and Operat...",2,1,NaN,NaN,11695,Hillsborough Community College
441,5202,"Business Administration, Management and Operat...",2,2,NaN,NaN,12044,Hillsborough Community College
442,5203,Accounting and Related Services.,2,1,NaN,NaN,14125,Hillsborough Community College
443,5207,Entrepreneurial and Small Business Operations.,2,1,NaN,NaN,17612,Hillsborough Community College


455 degrees availible. 100 have 5 year median income NaN. 345 remaining

In [ ]:
fields = ",".join([
    "school.name",

    "latest.programs.cip_4_digit.unit_id",
    "latest.programs.cip_4_digit.school.type",
    "latest.programs.cip_4_digit.school.main_campus",

    "programs.cip_4_digit.code",
    "programs.cip_4_digit.title",
    "programs.cip_4_digit.credential.level",
    "programs.cip_4_digit.distance",

    "latest.programs.cip_4_digit.earnings.4_yr.overall_median_earnings",
    "latest.programs.cip_4_digit.earnings.4_yr.working_not_enrolled.overall_count",
    "latest.programs.cip_4_digit.debt.staff_grad_plus.all.eval_inst.median"
])

params= {
    "api_key": SCORECARD_KEY,
    "school.state":"FL",
    "fields": fields,
    "latest.programs.cip_4_digit.earnings.4_yr.overall_median_earnings__not":"null",
    "latest.programs.cip_4_digit.debt.staff_grad_plus.all.eval_inst.median__not":"null",
    "page":"1",
    "per_page":"100"
}
response = requests.get(
    BASE_URL,
    params = params,
)
data2 = response.json()
df2 = pd.json_normalize(
    data2["results"],
    record_path=["latest.programs.cip_4_digit"],
    meta="school.name",
    errors="ignore"
)
df2

,code,title,unit_id,distance,school.type,school.main_campus,credential.level,earnings.4_yr.overall_median_earnings,earnings.4_yr.working_not_enrolled.overall_count,debt.staff_grad_plus.all.eval_inst.median,school.name
0,5106,Dental Support Services and Allied Professions.,133845,1,"Private, for-profit",1,1,26765,254,9500,Concorde Career Institute-Jacksonville
1,5107,Health and Medical Administrative Services.,133845,1,"Private, for-profit",1,1,29667,41,9500,Concorde Career Institute-Jacksonville
2,5108,Allied Health and Medical Assisting Services.,133845,1,"Private, for-profit",1,1,26428,216,9500,Concorde Career Institute-Jacksonville
3,5109,"Allied Health Diagnostic, Intervention, and Tr...",133845,1,"Private, for-profit",1,1,40294,73,15335,Concorde Career Institute-Jacksonville
4,5109,"Allied Health Diagnostic, Intervention, and Tr...",133845,1,"Private, for-profit",1,2,53958,58,20000,Concorde Career Institute-Jacksonville
...,...,...,...,...,...,...,...,...,...,...,...
347,4301,Criminal Justice and Corrections.,134608,3,Public,1,3,39116,71,13819,Indian River State College
348,4400,"Human Services, General.",134608,1,Public,1,3,33802,91,13500,Indian River State College
349,5138,"Registered Nursing, Nursing Administration, Nu...",134608,1,Public,1,2,48957,143,6500,Indian River State College
350,5138,"Registered Nursing, Nursing Administration, Nu...",134608,3,Public,1,3,78190,52,7250,Indian River State College


{'metadata': {'page': 1, 'total': 193, 'per_page': 20},
 'results': [{'latest.programs.cip_4_digit': [{'code': '5106',
     'title': 'Dental Support Services and Allied Professions.',
     'unit_id': 133845,
     'distance': 1,
     'school': {'type': 'Private, for-profit', 'main_campus': 1},
     'credential': {'level': 1},
     'earnings': {'4_yr': {'overall_median_earnings': 26765,
       'working_not_enrolled': {'overall_count': 254}}},
     'debt': {'staff_grad_plus': {'all': {'eval_inst': {'median': 9500}}}}},
    {'code': '5107',
     'title': 'Health and Medical Administrative Services.',
     'unit_id': 133845,
     'distance': 1,
     'school': {'type': 'Private, for-profit', 'main_campus': 1},
     'credential': {'level': 1},
     'earnings': {'4_yr': {'overall_median_earnings': 29667,
       'working_not_enrolled': {'overall_count': 41}}},
     'debt': {'staff_grad_plus': {'all': {'eval_inst': {'median': 9500}}}}},
    {'code': '5108',
     'title': 'Allied Health and Medic

352 degrees available with filtered 4 year median. we will use df2 moving forward

In [43]:
df2.isna().any()


code                                                False
title                                               False
unit_id                                             False
distance                                            False
school.type                                         False
school.main_campus                                  False
credential.level                                    False
earnings.4_yr.overall_median_earnings               False
earnings.4_yr.working_not_enrolled.overall_count    False
debt.staff_grad_plus.all.eval_inst.median           False
school.name                                         False
dtype: bool

In [46]:
df2.info()

<class 'pandas.DataFrame'>
MultiIndex: 352 entries, ('Concorde Career Institute-Jacksonville', 'Dental Support Services and Allied Professions.') to ('Indian River State College', 'Business Administration, Management and Operations.')
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype
---  ------                                            --------------  -----
 0   code                                              352 non-null    str  
 1   unit_id                                           352 non-null    int64
 2   distance                                          352 non-null    int64
 3   school.type                                       352 non-null    str  
 4   school.main_campus                                352 non-null    int64
 5   credential.level                                  352 non-null    int64
 6   earnings.4_yr.overall_median_earnings             352 non-null    int64
 7   earnings.4_yr.working_not_enrolled.

the earnings and debt features should be floats

In [44]:
df2=df2.set_index(["school.name","title"])

In [52]:
df2["debt.staff_grad_plus.all.eval_inst.median"] = df2["debt.staff_grad_plus.all.eval_inst.median"].astype(float)
df2["earnings.4_yr.working_not_enrolled.overall_count"] = df2["earnings.4_yr.working_not_enrolled.overall_count"].astype(float)
df2["earnings.4_yr.overall_median_earnings"] = df2["earnings.4_yr.overall_median_earnings"].astype(float)
df2.head()

code  \
school.name                            title                                                      
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.     5106   
                                       Health and Medical Administrative Services.         5107   
                                       Allied Health and Medical Assisting Services.       5108   
                                       Allied Health Diagnostic, Intervention, and Tre...  5109   
                                       Allied Health Diagnostic, Intervention, and Tre...  5109   

                                                                                           unit_id  \
school.name                            title                                                         
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.      133845   
                                       Health and Medical Administrative Services.          133845   
                                       Allied Health and Medical Assisting Services.        133845   
                                       Allied Health Diagnostic, Intervention, and Tre...   133845   
                                       Allied Health Diagnostic, Intervention, and Tre...   133845   

                                                                                           distance  \
school.name                            title                                                          
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.            1   
                                       Health and Medical Administrative Services.                1   
                                       Allied Health and Medical Assisting Services.              1   
                                       Allied Health Diagnostic, Intervention, and Tre...         1   
                                       Allied Health Diagnostic, Intervention, and Tre...         1   

                                                                                                   school.type  \
school.name                            title                                                                     
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.     Private, for-profit   
                                       Health and Medical Administrative Services.         Private, for-profit   
                                       Allied Health and Medical Assisting Services.       Private, for-profit   
                                       Allied Health Diagnostic, Intervention, and Tre...  Private, for-profit   
                                       Allied Health Diagnostic, Intervention, and Tre...  Private, for-profit   

                                                                                           school.main_campus  \
school.name                            title                                                                    
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.                      1   
                                       Health and Medical Administrative Services.                          1   
                                       Allied Health and Medical Assisting Services.                        1   
                                       Allied Health Diagnostic, Intervention, and Tre...                   1   
                                       Allied Health Diagnostic, Intervention, and Tre...                   1   

                                                                                           credential.level  \
school.name                            title                                                                  
Concorde Career Institute-Jacksonville Dental Support Services and Allied Professions.                    1   
                                    

In [53]:
df2.info()

<class 'pandas.DataFrame'>
MultiIndex: 352 entries, ('Concorde Career Institute-Jacksonville', 'Dental Support Services and Allied Professions.') to ('Indian River State College', 'Business Administration, Management and Operations.')
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   code                                              352 non-null    str    
 1   unit_id                                           352 non-null    int64  
 2   distance                                          352 non-null    int64  
 3   school.type                                       352 non-null    str    
 4   school.main_campus                                352 non-null    int64  
 5   credential.level                                  352 non-null    int64  
 6   earnings.4_yr.overall_median_earnings             352 non-null    float64
 7   earnings.4_yr.wor

In [55]:
df2.to_csv(r"..\data\raw\scorecard\test.csv")

In [63]:
df.index.get_level_values(0).nunique()

20